# Use pretrained model from feature extraction

In [1]:
from keras.applications import VGG16
conv_base=VGG16(weights="imagenet",
               include_top=False,
               input_shape=(150,150,3))

conv_base.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# Use VGG16 to extract features for the next Dense Layers

In [4]:
import os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

base_dir="small_train"
train_dir=os.path.join(base_dir,"train")
val_dir=os.path.join(base_dir,"val")
test_dir=os.path.join(base_dir,"test")
datagen=ImageDataGenerator(rescale=1/255.)
batch_size=20

def extract_features(dir_,sample_count):
    features=np.zeros((sample_count,4*4*512))
    labels=np.zeros((sample_count))
    generator=datagen.flow_from_directory(dir_,
                                         target_size=(150,150),
                                         batch_size=batch_size,
                                         class_mode="binary")
    i=0
    for data,label in generator:
        pred=conv_base.predict(data)
        features[i*batch_size:(i+1)*batch_size]=pred.reshape((batch_size,4*4*512))
        labels[i*batch_size:(i+1)*batch_size]=label
        i=i+1
        if i*batch_size>=sample_count:
            break
    return features,labels

train_features,train_labels=extract_features(train_dir,2000)
val_features,val_labels=extract_features(val_dir,1000)
test_features,test_labels=extract_features(test_dir,1000)

print("Train Shape:"+str(train_features.shape))
print("Val Shape:"+str(val_features.shape))
print("Test Shape:"+str(test_features.shape))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Train Shape:(2000, 8192)


# Build Dense Network 

In [5]:
from keras import models
from keras import layers
from keras import optimizers

model=models.Sequential()
model.add(layers.Dense(256,activation="relu",input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation="sigmoid"))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss="binary_crossentropy",
             metrics=["acc"])



In [6]:
history=model.fit(train_features,train_labels,
                 epochs=30,
                 batch_size=20,
                 validation_data=(val_features,val_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 549us/step - loss: 0.5901 - acc: 0.6800 - val_loss: 0.4802 - val_acc: 0.7720
Epoch 2/30
2000/2000 [==============================] - 1s 369us/step - loss: 0.4192 - acc: 0.8125 - val_loss: 0.3762 - val_acc: 0.8450
Epoch 3/30
2000/2000 [==============================] - 1s 380us/step - loss: 0.3522 - acc: 0.8500 - val_loss: 0.3437 - val_acc: 0.8570
Epoch 4/30
2000/2000 [==============================] - 1s 400us/step - loss: 0.3151 - acc: 0.8595 - val_loss: 0.3237 - val_acc: 0.8610
Epoch 5/30
2000/2000 [==============================] - 1s 385us/step - loss: 0.2836 - acc: 0.8855 - val_loss: 0.3121 - val_acc: 0.8610
Epoch 6/30
2000/2000 [==============================] - 1s 370us/step - loss: 0.2587 - acc: 0.9000 - val_loss: 0.3011 - val_acc: 0.8690
Epoch 7/30
2000/2000 [==============================] - 1s 404us/step - loss: 0.2471 - acc: 0.9020 - val_loss: 0.2936 - val_acc: 0.8750


# Test model

In [7]:
test_loss,test_acc=model.evaluate(x=test_features,y=test_labels)

1000/1000 [==============================] - 0s 136us/step


In [8]:
print("Testing loss:"+str(test_loss)+". Test acc:"+str(test_acc))

Testing loss:0.250557586551. Test acc:0.898
